In [ ]:
import json
import os
import re
from pathlib import Path

import fitz 
import pymupdf
import pymupdf.layout 
import pymupdf4llm
from markdown_it import MarkdownIt
import json
import unicodedata
import re

In [2]:
def boletim_to_markdown(path):
    md_text = pymupdf4llm.to_markdown(path)
    return md_text

md_text = boletim_to_markdown("BHA_PT_20260128.pdf")

OCR disabled because Tesseract language data not found.


In [24]:
lines = md_text.splitlines()
text_lines = []
for line in lines:
    if line != '' and "**==> picture" not in line:
        text_lines.append(line)

In [94]:
d_boletim = {}
# doi
doi = text_lines[0].split(":")[1].removesuffix("** ")
d_boletim['doi'] = doi
# issn
issn = text_lines[1].split(":")[1].lstrip().removesuffix("** ")
d_boletim['issn'] = issn
# volume
volume = text_lines[4].split()[1].removesuffix(",")
d_boletim['volume'] = volume
# number
number = text_lines[4].split()[3]
d_boletim['number'] = number
# date
date = " ".join(text_lines[4].split()[5:]).removesuffix("**")
d_boletim['date'] = date
# title
title = text_lines[6].removeprefix("## **").removesuffix("** ")
d_boletim['title'] = title
current_conditions = text_lines[66].removesuffix('**_ ')
d_boletim['current_conditions'] = {
    'text': current_conditions,
    'map_current_conditions': 'current_files/map_current_conditions.png',
    'table_current_conditions':'current_files/table_current_conditions.png'
    
    }


d_boletim

{'doi': '10.61818/02910604',
 'issn': '2965-0291',
 'volume': '6',
 'number': '04',
 'date': '28 de janeiro de 2026',
 'title': 'Boletim  de monitoramento climático de grandes bacias hidrográficas: Bacia Amazônica',
 'current_conditions': {'text': 'Mapas das condições observadas de precipitação e gráficos individuais por bacias são produzidos a partir dos dados MERGE/GPM gerados pelo INPE/CPTEC, considerando como climatologia para  período de 2000 a 2025. _**Entre os dias 30 de dezembro e 28 de janeiro  de  2026,  chuvas  abaixo  da  climatologia  caracterizaram  com  déficit de precipitação o curso principal do Rio Amazonas em territórios brasileiro e peruano, bacias hidrográficas dos rios Abacaxis, Aripuanã, Branco, Coari, Curuá Una, Guaporé, Içá, Iriri,  Javari,  Juruena,  Jutaí,  Mamoré,  bacias  da  margem  esquerda  do  Rio Amazonas no nordeste e no noroeste do Estado do Pará, Tapajós, Tefé, Teles Pires, Xingu e o curso principal do Rio Solimões; chuvas acima da climatologia regi

In [125]:
bacias = text_lines[71:143]
len(bacias)

72

In [140]:
bacias = []
for i in text_lines[71:145]:
    if "Página " not in i:
        bacias.append(i)
len(bacias)

64

In [167]:
bacias_names = []
for b in bacias[::2]:
    name = b.removeprefix("## _**").removeprefix("_**").removesuffix("**_ ")
    bacias_names.append(name)

md = MarkdownIt()
bacias_contents = []
for i in bacias[1::2]:
    md_render = md.render(i)
    bacias_contents.append(md_render)

In [168]:
dic_bacias = dict(zip(bacias_names, bacias_contents))

In [178]:
import unicodedata
import re

def normalizar_string(texto: str) -> str:

    # remove acentos
    texto = unicodedata.normalize("NFD", texto)
    texto = "".join(c for c in texto if unicodedata.category(c) != "Mn")

    # converte para minúsculas
    texto = texto.lower()

    # remove caracteres especiais (mantém letras e números)
    texto = re.sub(r"[^a-z0-9\s]", "", texto)

    # remove espaços extras
    texto = re.sub(r"\s+", " ", texto).strip()
    texto = texto.replace(" ", "-")

    return texto

x = normalizar_string('Bacia do Rio Marañon')
x

'bacia-do-rio-maranon'

In [202]:
analysis = []
for k, v in dic_bacias.items():
    slugfy = normalizar_string(k)
    item = {
        "id": slugfy,
        "name": k,
        "content": v,
        "charts": {
            "acc": f"{slugfy}-acc.png",
            "ano": f"{slugfy}-ano.png"
        }

    }
    analysis.append(item)


In [232]:
d_boletim['analysis'] = analysis

In [231]:
for i in analysis:
    content = i['content']
    strong = content.split('<em><strong>')
    climatologia = strong[1].split("</strong></em>")[0]
    i['climatologia'] = climatologia
    observados = strong[3].split("</strong></em>")[0]
    i['observados'] = observados
    anomalia = strong[4].split("</strong></em>")[0].removesuffix(",")
    i['anomalia'] = anomalia
    classification = strong[5].split("</strong></em>")[0].removesuffix(".")
    i['classification'] = classification
    prognostico = strong[-1].split("</strong></em>")[0].removesuffix(".")
    i['prognostico'] = prognostico
    print(prognostico)

próximo da normalidade ou tendência a chuvoso
chuvoso ou tendência a chuvoso
chuvoso  ou  tendência  a muito chuvoso
chuvoso ou tendência a muito chuvoso
próximo  da normalidade ou tendência a chuvoso
seco ou tendência a seco
seco ou tendência a seco
próximo da normalidade
seco ou tendência a muito seco
próximo  da normalidade ou tendência a chuvoso
próximo  da normalidade
próximo da normalidade
próximo da normalidade ou tendência a seco
chuvoso ou tendência a chuvoso
próximo da normalidade
próximo  da normalidade ou tendência a chuvoso
seco ou tendência a muito seco
seco ou tendência a muito seco
próximo  da normalidade
próximo da normalidade ou tendência a seco
chuvoso  ou tendência a chuvoso
próximo  da normalidade
próximo da normalidade
seco ou tendência a seco
seco ou tendência a seco
seco ou tendência a seco
seco ou tendência a muito seco
muito seco ou tendência a muito seco
muito seco ou tendência a extremamente seco
seco ou tendência a seco
seco ou tendência a muito seco
muito 

In [234]:
with open('../data/current_boletim.json', 'w') as f:
    json.dump(d_boletim, f, indent=3, ensure_ascii=False)

In [18]:
print(lines)

['**DOI:10.61818/02910604** ', '', '**==> picture [102 x 102] intentionally omitted <==**', '', '# **ISSN: 2965-0291** ', '', '**==> picture [117 x 116] intentionally omitted <==**', '', '# **Boletim** ', '', '# **de monitoramento climático de grandes bacias hidrográficas: Bacia Amazônica** ', '', '**Volume 6, Número 04                              Manaus, 28 de janeiro de 2026** ', '', '**==> picture [483 x 96] intentionally omitted <==**', '', 'i ', '', '## **Boletim  de monitoramento climático de grandes bacias hidrográficas: Bacia Amazônica** ', '', '**Editor Chefe Renato Cruz Senna** ', '', '**Meteorologista** ', '', '   - **Pesquisador - CODAM, INPA** ', '', '- **Editoração Renato Cruz Senna** ', '', '   - **Tainá Sampaio Xavier Conchy Rocha** ', '', '**Adriano Nobre Arcos** ', '', '   - **Periodicidade Semanal** ', '', '- **Revisão e Diagramação Inácio de Oliveira Lima Neto** ', '', '      - **Contato Instituto Nacional de Pesquisas da Amazônia - INPA** ', '', '**Av. André Araúj

In [62]:
doc = pymupdf.open('BHA_PT_20260128.pdf')

# Imagens

In [193]:
analysis[0]

{'id': 'bacia-do-rio-branco',
 'name': 'Bacia do Rio Branco',
 'content': '<p>A climatologia do período em análise indica chuvas com registros variando entre <em><strong>39 e 51 mm</strong></em> sendo considerados normais (referência quantis 42.5% e 57.5%). Em <em><strong>28 de janeiro  de  2026,</strong></em> foram  observados <em><strong>29 mm</strong></em> de precipitação média acumulada sobre a bacia em 30 dias, o cálculo da média do índice de  anomalia  categorizada na  área da  bacia  o  valor de <em><strong>-0.9,</strong></em> classifica a bacia em condição de <em><strong>tendência a seco.</strong></em> Nas próximas semanas o comportamento  climático  indica <em><strong>manutenção</strong></em> dos volumes de chuva, o modelo de prognóstico subsazonal sugere um comportamento <em><strong>próximo da normalidade ou tendência a chuvoso.</strong></em></p>\n'}

In [196]:
images = []
for i in analysis:
    id = i['id']
    images.append(f'{id}-acc.png')
    images.append(f'{id}-ano.png')
len(images)

64

In [201]:
c = 0
for i in range(4, 15):
    page = doc.load_page(i)
    page_dict = page.get_text("dict") 
    blocks = page_dict.get("blocks", [])
    logo = (175.0500030517578, 776.2003173828125, 457.0, 832.2003173828125)
    for b in blocks:
        btype = b.get("type", None)  
        bbox = b.get("bbox", None)
        if btype == 1 and bbox != logo:
            rect = fitz.Rect(bbox)
            pix = page.get_pixmap(clip=rect, dpi=200, alpha=False)
            img = images[c]
            pix.save(f'img/{img}')
            print(img)
            c += 1

bacia-do-rio-branco-acc.png
bacia-do-rio-branco-ano.png
bacia-do-rio-negro-acc.png
bacia-do-rio-negro-ano.png
bacia-do-rio-maranon-acc.png
bacia-do-rio-maranon-ano.png
bacia-do-rio-ucayali-acc.png
bacia-do-rio-ucayali-ano.png
bacia-do-rio-napo-acc.png
bacia-do-rio-napo-ano.png
curso-principal-do-rio-amazonas-peru-acc.png
curso-principal-do-rio-amazonas-peru-ano.png
bacia-do-rio-javari-acc.png
bacia-do-rio-javari-ano.png
bacia-do-rio-ica-putumayo-acc.png
bacia-do-rio-ica-putumayo-ano.png
bacia-do-rio-jutai-acc.png
bacia-do-rio-jutai-ano.png
bacia-do-rio-jurua-acc.png
bacia-do-rio-jurua-ano.png
bacia-do-rio-japura-caqueta-acc.png
bacia-do-rio-japura-caqueta-ano.png
bacia-do-rio-tefe-acc.png
bacia-do-rio-tefe-ano.png
bacia-do-rio-coari-acc.png
bacia-do-rio-coari-ano.png
bacia-do-rio-purus-acc.png
bacia-do-rio-purus-ano.png
curso-principal-do-rio-solimoes-acc.png
curso-principal-do-rio-solimoes-ano.png
bacia-dos-rios-beni-e-madre-de-dios-acc.png
bacia-dos-rios-beni-e-madre-de-dios-ano.png


In [189]:
img = 1
page = doc.load_page(4)
page_dict = page.get_text("dict") 
blocks = page_dict.get("blocks", [])
logo = (175.0500030517578, 776.2003173828125, 457.0, 832.2003173828125)
for b in blocks:
    btype = b.get("type", None)  # 0 = texto, 1 = imagem
    bbox = b.get("bbox", None)
    if btype == 1 and bbox != logo:
        rect = fitz.Rect(bbox)
        pix = page.get_pixmap(clip=rect, dpi=200, alpha=False)
        pix.save(f'img/{img}.png')
        print(bbox, img)
        img += 1
        

(75.5999984741211, 98.25035095214844, 248.39999389648438, 199.05035400390625) 1
(77.05000305175781, 205.55035400390625, 249.85000610351562, 301.30035400390625) 2
(75.69999694824219, 339.65032958984375, 248.5, 440.4503173828125) 3
(77.05000305175781, 439.0003662109375, 249.85000610351562, 534.7503662109375) 4
(75.69999694824219, 579.850341796875, 248.5, 680.6503295898438) 5
(77.05000305175781, 684.350341796875, 249.85000610351562, 780.100341796875) 6


In [95]:
def extrac_img_page3(page):
    # mapa
    x0, y0, x1, y1 = 136.3000030517578, 414.7003479003906, 483.6500244140625, 682.5503540039062  
    rect = fitz.Rect(x0, y0, x1, y1)
    zoom = 3 
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, clip=rect, alpha=False)
    pix.save(f'current_files/map_current_conditions.png')
    # table
    x0, y0, x1, y1 = 100, 680, 515, 765   
    rect = fitz.Rect(x0, y0, x1, y1)
    zoom = 3 
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, clip=rect, alpha=False)
    pix.save(f'current_files/table_current_conditions.png')

extrac_img_page3(page)

# Page 1

In [8]:
page = doc.load_page(0)
page_dict = page.get_text("dict") 
blocks = page_dict.get("blocks", [])

page 0 of BHA_PT_20260128.pdf

In [62]:
def get_doi(doc, boletim_dict):
    page = doc.load_page(0)
    page_dict = page.get_text("dict") 
    blocks = page_dict.get("blocks", [])
    text_list = []
    for b in blocks:
        btype = b.get("type", None) 
        bbox = b.get("bbox", None)
        if btype == 0:
            for line in b.get("lines", []):
                line_text = "".join(span.get("text", "") for span in line.get("spans", []))
                if line_text.strip():
                    text_list.append(line_text)
    doi = text_list[0].split(':')[1]
    issn = text_list[1].split(":")[1].strip()
    title = "".join(text_list[2:-1]).replace(":", ": ")
    volume = text_list[-1].split()[1]
    number = text_list[-1].split()[3]
    date = " ".join(text_list[-1].split()[5:])
    boletim_dict['doi'] = doi
    boletim_dict['issn'] = issn
    boletim_dict['title'] = title
    boletim_dict['volume'] = volume
    boletim_dict['number'] = number
    boletim_dict['date'] = date

    return boletim_dict
d = {}
d = get_doi(doc, d)
d

{'doi': '10.61818/02910604',
 'issn': '2965-0291',
 'title': 'Boletim de monitoramento climático de grandes bacias hidrográficas: Bacia Amazônica',
 'volume': '6,',
 'number': '04',
 'date': '28 de janeiro de 2026'}

# Page 3

In [69]:
page = doc.load_page(3)
page_dict = page.get_text("dict") 
blocks = page_dict.get("blocks", [])

# Make json

In [92]:
with open("output.md", "r", encoding="utf-8") as f:
    content = f.read()

md = MarkdownIt()
tokens = md.parse(content)

In [100]:
tokens[1]

Token(type='inline', tag='', nesting=0, attrs={}, map=[0, 1], level=1, children=[Token(type='text', tag='', nesting=0, attrs={}, map=None, level=0, children=None, content='', markup='', info='', meta={}, block=False, hidden=False), Token(type='strong_open', tag='strong', nesting=1, attrs={}, map=None, level=0, children=None, content='', markup='**', info='', meta={}, block=False, hidden=False), Token(type='text', tag='', nesting=0, attrs={}, map=None, level=1, children=None, content='DOI:10.61818/02910604 ISSN: 2965-0291', markup='', info='', meta={}, block=False, hidden=False), Token(type='strong_close', tag='strong', nesting=-1, attrs={}, map=None, level=0, children=None, content='', markup='**', info='', meta={}, block=False, hidden=False), Token(type='text', tag='', nesting=0, attrs={}, map=None, level=0, children=None, content='', markup='', info='', meta={}, block=False, hidden=False)], content='**DOI:10.61818/02910604 ISSN: 2965-0291**', markup='', info='', meta={}, block=True, 

In [101]:
data = []
current_section = None
for token in tokens:
    if token.type == "heading_open":
        level = int(token.tag[1])
        current_section = {
            "level": level,
            "title": "",
            "content": []
        }
        data.append(current_section)

    elif token.type == "inline" and current_section:
        if current_section["title"] == "":
            current_section["title"] = token.content
        else:
            current_section["content"].append(token.content)

In [184]:
boletim_dic = {}
for d in data:
    if d['level'] == 1:
        title = d['title'].replace("**", "")
        boletim_dic['title'] = title
    elif d['level'] == 2:
        text = d['title'].split(":")
        doi = text[1].split()[0]
        issn = text[-1].removesuffix("**")
        boletim_dic['doi'] = doi
        boletim_dic['issn'] = issn
    elif d['level'] == 3:
        if d['title'] == '**Índice**':
            con = d['content']
            for i in con:
                if "_**Condições atuais**_" in i:
                    index = con.index(i)
                    condicoes = con[index+1]
                    boletim_dic['condicoes_atuais'] = condicoes.replace("**_", "").replace("_**", "").replace("\n", " ")
                elif i == "_**Bacia do Rio Branco**_":
                    index = con.index(i)
                    bacias = con[index+1:]
                    # print(con[index:])
    elif d['level'] == 4:
        # print(d)
        pass
    elif d['level'] == 5:
        text = d['title'].replace("**", "").split()
        boletim_dic['volume'] = text[1]
        boletim_dic['numero'] = text[3]
        boletim_dic['date'] = " ".join(text[5:])
        
boletim_dic

{'doi': '10.61818/02910604',
 'issn': ' 2965-0291',
 'title': 'Boletim de monitoramento climático de grandes bacias hidrográficas: Bacia Amazônica',
 'volume': '6,',
 'numero': '04',
 'date': '28 de janeiro de 2026',
 'condicoes_atuais': 'Mapas das condições observadas de precipitação e gráficos individuais por bacias são produzidos a partir dos dados MERGE/GPM gerados pelo INPE/CPTEC, considerando como climatologia para período de 2000 a 2025. Entre os dias 30 de dezembro e 28 de janeiro de 2026, chuvas abaixo da climatologia caracterizaram com déficit de precipitação o curso principal do Rio Amazonas em territórios brasileiro e peruano, bacias hidrográficas dos rios Abacaxis, Aripuanã, Branco, Coari, Curuá Una, Guaporé, Içá, Iriri, Javari, Juruena, Jutaí, Mamoré, bacias da margem esquerda do Rio Amazonas no nordeste e no noroeste do Estado do Pará, Tapajós, Tefé, Teles Pires, Xingu e o curso principal do Rio Solimões; chuvas acima da climatologia registradas sobre as bacias dos rios 

In [179]:
bacias_text = " ".join(bacias)
rio_branco = bacias_text.split("_**Bacia do Rio Negro**_")[0]
bacias_text = bacias_text.split("_**Bacia do Rio Negro**_")[1]
rio_negro = bacias_text.split("_**Bacia do Rio Marañon**_")[0]
bacias_text = bacias_text.split("_**Bacia do Rio Marañon**_")[1]
RioMaranon = bacias_text.split("_**Bacia do Rio Ucayali**_")[0]
bacias_text = bacias_text.split("_**Bacia do Rio Ucayali**_")[1]

In [182]:
print(rio_branco)

A climatologia do período em análise indica chuvas com
registros variando entre _**39 e 51 mm**_ sendo considerados
normais (referência quantis 42.5% e 57.5%). Em _**28 de**_
_**janeiro de 2026,**_ foram observados _**29 mm**_ de
precipitação média acumulada sobre a bacia em 30 dias, na área da bacia o valor de _**-0.9,**_ classifica a bacia em
condição de _**tendência a seco.**_ Nas próximas semanas o
comportamento climático indica _**manutenção**_ dos
volumes de chuva, o modelo de prognóstico subsazonal
sugere um comportamento _**próximo da normalidade ou**_
_**tendência a chuvoso.**_ 


In [186]:
analise_individual = [{'bacia': 'Rio Branco', 'content': md.render(rio_branco) }]
boletim_dic['analise_individual'] = analise_individual

In [187]:
boletim_dic

{'doi': '10.61818/02910604',
 'issn': ' 2965-0291',
 'title': 'Boletim de monitoramento climático de grandes bacias hidrográficas: Bacia Amazônica',
 'volume': '6,',
 'numero': '04',
 'date': '28 de janeiro de 2026',
 'condicoes_atuais': 'Mapas das condições observadas de precipitação e gráficos individuais por bacias são produzidos a partir dos dados MERGE/GPM gerados pelo INPE/CPTEC, considerando como climatologia para período de 2000 a 2025. Entre os dias 30 de dezembro e 28 de janeiro de 2026, chuvas abaixo da climatologia caracterizaram com déficit de precipitação o curso principal do Rio Amazonas em territórios brasileiro e peruano, bacias hidrográficas dos rios Abacaxis, Aripuanã, Branco, Coari, Curuá Una, Guaporé, Içá, Iriri, Javari, Juruena, Jutaí, Mamoré, bacias da margem esquerda do Rio Amazonas no nordeste e no noroeste do Estado do Pará, Tapajós, Tefé, Teles Pires, Xingu e o curso principal do Rio Solimões; chuvas acima da climatologia registradas sobre as bacias dos rios 

In [190]:
with open('output.json', 'w', encoding='utf-8') as j:
    json.dump(boletim_dic, j, indent=2, ensure_ascii=False)

In [180]:
bacias_text

' _**Bacia do Rio Napo**_ _**Bacia Amazônica – CODAM  Página 3**_ A climatologia do período em análise indica chuvas com\nregistros variando entre _**171 e 193 mm**_ sendo\nconsiderados normais (referência quantis 42.5% e\n57.5%). Em _**28 de janeiro de 2026,**_ foram observados\n_**212 mm**_ de precipitação média acumulada sobre a bacia\nem 30 dias, no cálculo da média do índice de anomalia\ncategorizada na área da bacia o valor de _**0.8,**_ classifica\na bacia em condição de _**tendência a chuvoso.**_ Nas\npróximas semanas o comportamento climático indica\n_**manutenção**_ dos volumes de chuva, o modelo de\nprognóstico subsazonal sugere um comportamento\n_**chuvoso ou tendência a muito chuvoso.**_ A climatologia do período em análise indica chuvas com\nregistros variando entre _**182 e 219 mm**_ sendo\nconsiderados normais (referência quantis 42.5% e\n57.5%). Em _**28 de janeiro de 2026,**_ foram observados\n_**201 mm**_ de precipitação média acumulada sobre a bacia\nem 30 dias, no 

In [173]:
for b in bacias:
    print(b)

A climatologia do período em análise indica chuvas com
registros variando entre _**39 e 51 mm**_ sendo considerados
normais (referência quantis 42.5% e 57.5%). Em _**28 de**_
_**janeiro de 2026,**_ foram observados _**29 mm**_ de
precipitação média acumulada sobre a bacia em 30 dias,
na área da bacia o valor de _**-0.9,**_ classifica a bacia em
condição de _**tendência a seco.**_ Nas próximas semanas o
comportamento climático indica _**manutenção**_ dos
volumes de chuva, o modelo de prognóstico subsazonal
sugere um comportamento _**próximo da normalidade ou**_
_**tendência a chuvoso.**_
_**Bacia do Rio Negro**_
A climatologia do período em análise indica chuvas com
registros variando entre _**167 e 194 mm**_ sendo
considerados normais (referência quantis 42.5% e
57.5%). Em _**28 de janeiro de 2026,**_ foram observados
_**203 mm**_ de precipitação média acumulada sobre a bacia
em 30 dias, no cálculo da média do índice de anomalia
categorizada na área da bacia o valor de _**0.5,**_ class

Mapas das condições observadas de precipitação e gráficos individuais por bacias são
produzidos a partir dos dados MERGE/GPM gerados pelo INPE/CPTEC, considerando
como climatologia para período de 2000 a 2025. _**Entre os dias 30 de dezembro e 28 de**_
_**janeiro de 2026, chuvas abaixo da climatologia caracterizaram com déficit de**_
_**precipitação o curso principal do Rio Amazonas em territórios brasileiro e peruano,**_
_**bacias hidrográficas dos rios Abacaxis, Aripuanã, Branco, Coari, Curuá Una, Guaporé,**_
_**Içá, Iriri, Javari, Juruena, Jutaí, Mamoré, bacias da margem esquerda do Rio**_
_**Amazonas no nordeste e no noroeste do Estado do Pará, Tapajós, Tefé, Teles Pires,**_
_**Xingu e o curso principal do Rio Solimões; chuvas acima da climatologia registradas**_
_**sobre as bacias dos rios Marañon, Negro e Ucayali; chuvas próximas da normalidade**_
_**registradas sobre as bacias hidrográficas dos Beni, Japurá, Ji-Paraná, Juruá, Madeira,**_
_**bacias da margem esquerda do Rio Amazonas no nordeste do Estado do Amazonas,**_
_**Napo e Purus. O multimodelo indica para as próximas semanas previsão de chuvas**_
_**abaixo da climatologia no leste, sudeste e sul da região monitorada, sobre o curso**_
_**principal do Rio Amazonas em território brasileiro, bacias dos rios Abacaxis,**_
_**Aripuanã, Beni, Curuá Una, Guaporé, Iriri, Ji-Paraná, Juruena, Mamoré, bacias da**_
_**margem esquerda do Rio Amazonas no nordeste do Estado do Amazonas e no**_
_**nordeste e no noroeste do Estado do Pará, Tapajós, Teles Pires e Xingu; previsão de**_
_**chuvas acima da climatologia no noroeste da região monitorada, sobre o curso**_
_**principal do Rio Amazonas em território peruano, bacias dos rios Içá, Japurá,**_
_**Marañon, Napo e Negro.**_